In [4]:
import _init_paths

import tensorflow as tf
import numpy as np
import nengo_dl
import nengo 

from configs.exp_configs import tf_exp_cfg as tf_cfg, nengo_dl_cfg as ndl_cfg
from utils.cnn_2d_utils import get_2d_cnn_model
from utils.nengo_dl_utils import get_nengo_dl_model
from utils.base_utils.exp_utils import get_grouped_slices_2d_pooling
from utils.base_utils.data_prep_utils import get_batches_of_exp_dataset, get_exp_dataset

In [2]:
model = get_2d_cnn_model((3, 32, 32), tf_cfg) # Channels first shape (1, 28, 28).
model[0].summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 32, 32)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 3, 32, 32)         9         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 30, 30)         216       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 15, 15)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 13, 13)        1152      
_________________________________________________________________
flatten (Flatten)            (None, 2704)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                173120

In [3]:
ndl_model = nengo_dl.Converter(model[0])

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:588: UserWarning: Activation type <function softmax at 0x2ab9d2560ef0> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


In [5]:
print(dir(ndl_model))

['KerasTensorDict', 'TrackedDict', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_layer_converters', 'allow_fallback', 'converters', 'get_converter', 'inference_only', 'inputs', 'layers', 'max_to_avg_pool', 'model', 'net', 'outputs', 'register', 'scale_firing_rates', 'split_shared_weights', 'swap_activations', 'synapse', 'temporal_model', 'verify']


In [8]:
for lyr in ndl_model.model.layers:
  if lyr.name in ["conv2d", "conv2d_1"]:
    print(lyr.output.shape[1:])

(16, 26, 26)
(24, 11, 11)


In [20]:
nengo_dl.__version__

'3.4.0'

In [2]:
grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=1, rows=5, cols=5)

In [3]:
len(grouped_slices) # 10816 == 16 x 26 x 26

25

In [6]:
grouped_slices[:16]

array([ 0,  1,  5,  6,  2,  3,  7,  8, 10, 11, 15, 16, 12, 13, 17, 18])

In [5]:
grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=24, rows=11, cols=11)

In [6]:
len(grouped_slices)# 24 x 11 x 11 = 2904 

# ###############################################################

In [2]:
ndl_model, ngo_probes_lst = get_nengo_dl_model(
    (1, 28, 28), tf_cfg, ndl_cfg, mode="test", num_clss=10, max_to_avg_pool=False)

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:588: UserWarning: Activation type <function softmax at 0x2ae894efbb90> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


In [8]:
ndl_model.net.all_connections[3].post_obj.size_in

21632

In [4]:
for i, conn in enumerate(ndl_model.net.all_connections):
  if (isinstance(conn.post_obj, nengo_dl.tensor_node.TensorNode) and
      conn.post_obj.label.startswith("max_pooling")):
    print(conn, ndl_model.net.all_connections[i])

<Connection from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">> <Connection from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">>
<Connection from <Neurons of <Ensemble "conv2d_1.0">> to <TensorNode "max_pooling2d_1">> <Connection from <Neurons of <Ensemble "conv2d_1.0">> to <TensorNode "max_pooling2d_1">>


In [5]:
ndl_model.model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 26, 26)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 13, 13)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 11, 11)        3480      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 5, 5)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 3, 3)          6944      
_________________________________________________________________
flatten (Flatten)            (None, 288)               0     

In [2]:
def get_grouped_slices_2d_pooling(**kwargs):
  """
  Creates square grouped slices based on the `pool_size`. E.g. for a flattened
  array, the indices are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
  ...] with `pool_size` = (2, 2), `rows=4`, `cols=4`, the flattened array is
  actually:
  [[0, 1, 2, 3],
   [4, 5, 6, 7],
   [8, 9, 10, 11],
   [12, 13, 14, 15]]

  so the returned grouped slices array should be of indices:
  [0, 1, 4, 5, 2, 3, 6, 7, 8, 9, 12, 13, 10, 11, 14, 15, ...]

  Note: It expects the "Channels First" coding of Input/Conv layers. It is also
  assumed that the input `rows` and `cols` are of same size and `pool_size` also
  has same row/col dimension.

  Args:
    kwargs <dict>:
      pool_size <tuple>: (int, int) for 2D Pooling - (row, col) arrangement.
      num_chnls <int>: Number of channels in the reshaped matrix.
      rows <int>: Number of rows in the reshaped matrix.
      cols <int>: Number of columns in the reshaped matrix.

  Returns:
    <[int]>
  """
  pool_size, num_chnls, rows, cols = (
      kwargs["pool_size"], kwargs["num_chnls"], kwargs["rows"], kwargs["cols"])
  matrix = np.arange(rows * cols * num_chnls).reshape((num_chnls, rows, cols))
  grouped_slices = np.zeros(rows * cols * num_chnls, dtype=int)
  start, slice_len = 0, np.prod(pool_size)

  for chnl in range(num_chnls):
    for row in range(rows//pool_size[0]):
      for col in range(cols//pool_size[1]):
        grouped_slices[start:start+slice_len] = (
            matrix[chnl, row*pool_size[0]:row*pool_size[0]+pool_size[0],
                   col*pool_size[1]:col*pool_size[1]+pool_size[1]]).flatten()
        start += slice_len
        
  return grouped_slices

In [3]:
get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=4, rows=3, cols=3)

array([ 0,  1,  3,  4,  9, 10, 12, 13, 18, 19, 21, 22, 27, 28, 30, 31,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [26]:
def get_grouped_slices_2d_pooling(is_channels_last=False, **kwargs):
  """
  Creates square grouped slices based on the `pool_size`. E.g. for a flattened
  array, the indices are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
  ...] with `pool_size` = (2, 2), `rows=4`, `cols=4`, the flattened array is
  actually:
  [[0, 1, 2, 3],
   [4, 5, 6, 7],
   [8, 9, 10, 11],
   [12, 13, 14, 15]]

  so the returned grouped slices array should be of indices:
  [0, 1, 4, 5, 2, 3, 6, 7, 8, 9, 12, 13, 10, 11, 14, 15, ...]

  Note: It expects the "Channels First" coding of Input/Conv layers. It is also
  assumed that the input `rows` and `cols` are of same size and `pool_size` also
  has same row/col dimension.

  Args:
    kwargs <dict>:
      pool_size <tuple>: (int, int) for 2D Pooling - (row, col) arrangement.
      num_chnls <int>: Number of channels in the reshaped matrix.
      rows <int>: Number of rows in the reshaped matrix.
      cols <int>: Number of columns in the reshaped matrix.

  Returns:
    <[int]>
  """
  pool_size, num_chnls, rows, cols = (
      kwargs["pool_size"], kwargs["num_chnls"], kwargs["rows"], kwargs["cols"])
  matrix = np.arange(rows * cols * num_chnls).reshape((rows, cols, num_chnls))
#   if is_channels_last:
#     matrix = np.moveaxis(matrix, 0, -1)
  grouped_slices = np.zeros(rows * cols * num_chnls, dtype=int)
  start, slice_len = 0, np.prod(pool_size)

  for chnl in range(num_chnls):
    for row in range(rows//pool_size[0]):
      for col in range(cols//pool_size[1]):
#         if is_channels_last:
#           grouped_slices[start:start+slice_len] = (
#               matrix[row*pool_size[0]:row*pool_size[0]+pool_size[0],
#               col*pool_size[1]:col*pool_size[1]+pool_size[1], chnl]).flatten()
#         else:
          grouped_slices[start:start+slice_len] = (
              matrix[row*pool_size[0]:row*pool_size[0]+pool_size[0],
              col*pool_size[1]:col*pool_size[1]+pool_size[1], chnl]).flatten()
          start += slice_len
        
  return grouped_slices

In [28]:
cl_grouped_slices = get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=4, rows=3, cols=3)
cl_grouped_slices

array([ 0,  4, 12, 16,  1,  5, 13, 17,  2,  6, 14, 18,  3,  7, 15, 19,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [7]:
get_grouped_slices_2d_pooling(pool_size=(2, 2), num_chnls=4, rows=3, cols=3, is_channels_last=True)

array([ 0,  1,  3,  4,  9, 10, 12, 13, 18, 19, 21, 22, 27, 28, 30, 31,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [16]:
matrix_cl = np.moveaxis(matrix, 0, -1)
print(matrix_cl.shape)
print(matrix_cl)

(3, 3, 4)
[[[ 0  9 18 27]
  [ 1 10 19 28]
  [ 2 11 20 29]]

 [[ 3 12 21 30]
  [ 4 13 22 31]
  [ 5 14 23 32]]

 [[ 6 15 24 33]
  [ 7 16 25 34]
  [ 8 17 26 35]]]


In [19]:
flattened = matrix_cl.flatten()
flattened

array([ 0,  9, 18, 27,  1, 10, 19, 28,  2, 11, 20, 29,  3, 12, 21, 30,  4,
       13, 22, 31,  5, 14, 23, 32,  6, 15, 24, 33,  7, 16, 25, 34,  8, 17,
       26, 35])

In [29]:
flattened[cl_grouped_slices]

array([ 0,  1,  3,  4,  9, 10, 12, 13, 18, 19, 21, 22, 27, 28, 30, 31,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])

In [30]:
reshaped_matrix_cl = np.reshape(flattened, (3, 3, 4))
reshaped_matrix_cl

array([[[ 0,  9, 18, 27],
        [ 1, 10, 19, 28],
        [ 2, 11, 20, 29]],

       [[ 3, 12, 21, 30],
        [ 4, 13, 22, 31],
        [ 5, 14, 23, 32]],

       [[ 6, 15, 24, 33],
        [ 7, 16, 25, 34],
        [ 8, 17, 26, 35]]])